# Random Forest Parkinsons Approach 


In [174]:
# Libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Training Data
train_clinical = './train_clinical_data.csv'
train_peptides = './train_peptides.csv'
train_supp_clinical = './supplemental_clinical_data.csv'
train_proteins = './train_proteins.csv'

# Given Test Data
test_proteins= './example_test_files/test_proteins.csv'
test_peptides = './example_test_files/test_peptides.csv'
test_clinical = './example_test_files/test.csv'

## Load Training and Test Data

In [175]:
clinical_data = pd.read_csv(train_clinical)
protein_data = pd.read_csv(train_proteins)
peptide_data = pd.read_csv(train_peptides)
supp_data = pd.read_csv(train_supp_clinical)


clinical_data['upd23b_clinical_state_on_medication'] = clinical_data['upd23b_clinical_state_on_medication'].replace({'On': 1})
clinical_data['upd23b_clinical_state_on_medication'] = clinical_data['upd23b_clinical_state_on_medication'].replace({'Off': -1})
clinical_data['upd23b_clinical_state_on_medication'] = clinical_data['upd23b_clinical_state_on_medication'].fillna(0)

# Naming convention for me to use w/o ruining codyʻs code
train_proteins = protein_data
train_peptides = peptide_data
train_clinical = clinical_data

test_proteins = pd.read_csv(test_proteins)
test_peptides = pd.read_csv(test_peptides)
test_clinical = pd.read_csv(test_clinical)

## Imported Data

In [176]:
# Protein Training Data
print(train_proteins.shape)
train_proteins.head()

(232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [177]:
# Protein Testing Data
print(test_proteins.shape)
test_proteins.head()

(453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [178]:
# Peptide Training Data
print(train_peptides.shape)
train_peptides.head()

(981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [179]:
# Peptide Testing Data
print(test_peptides.shape)
test_peptides.head()

(2057, 7)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.3,0
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.6,0
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.4,0
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.0,0
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.2,0


In [180]:
# CLinical Training Data
print(train_clinical.shape)
train_clinical.head()

(2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,0.0
1,55_3,55,3,10.0,7.0,25.0,NaN,0.0
2,55_6,55,6,8.0,10.0,34.0,NaN,0.0
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0


In [181]:
# CLinical Testing Data
print(test_clinical.shape)
test_clinical.head()

(16, 6)


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


## Understanding The Data

In [182]:
train_proteins.groupby('visit_id')
train_proteins

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [183]:
# Organize by visit_id (which includes the patient_id + visit_month)
# Protein data is aggregated based on information that is useful for me (# unique proteins, details about the NPX scores)
# Peptide data also include # unique peptides (multiple peptides make up a protein)
'''
Note: High NPX score means high protein concentration. If we find what proteins in parkinsons patients have high
concentrations, we can assume they are more correlated to the disease being present
'''
agg_protein_data = train_proteins.groupby('visit_id').agg({'UniProt':'nunique','NPX':['min','max','mean','std']}).reset_index()
agg_peptide_data = train_peptides.groupby('visit_id').agg({'UniProt':'nunique','Peptide':'nunique','PeptideAbundance': ['min','max','mean','std']}).reset_index()

In [184]:
agg_protein_data
# agg_protein_data.loc[agg_protein_data['visit_id'] == '55_0']

visit_id UniProt       NPX                                         
               nunique       min          max          mean           std
0      10053_0     165  2497.840  269126000.0  2.856580e+06  2.131630e+07
1     10053_12     171  5800.870  270030000.0  2.728871e+06  2.092162e+07
2     10053_18     208  1334.110  278835000.0  2.509967e+06  1.969453e+07
3     10138_12     217  2520.240  365582000.0  3.002583e+06  2.516170e+07
4     10138_24     219  1436.940  396894000.0  3.068891e+06  2.716806e+07
...        ...     ...       ...          ...           ...           ...
1108   8699_24     216   756.551  346067000.0  3.064059e+06  2.409420e+07
1109    942_12     212  1722.770  330558000.0  2.613298e+06  2.295228e+07
1110    942_24     217  1339.150  336769000.0  2.616142e+06  2.312662e+07
1111    942_48     216  1272.480  358059000.0  2.768442e+06  2.460543e+07
1112     942_6     216  2491.690  290111000.0  2.476880e+06  2.002174e+07

[1113 rows x 6 columns]

In [185]:
agg_peptide_data

visit_id UniProt Peptide PeptideAbundance                              \
               nunique nunique              min          max           mean   
0      10053_0     165     649          82.9679   66333900.0  726248.393431   
1     10053_12     171     633         128.4460   73059300.0  737183.385744   
2     10053_18     208     868         108.5000   64711200.0  601466.784320   
3     10138_12     217     932         129.0240   71652400.0  699099.199189   
4     10138_24     219     918         142.6480  123897000.0  732120.888877   
...        ...     ...     ...              ...          ...            ...   
1108   8699_24     216     911         106.9420   99846400.0  726494.824901   
1109    942_12     212     889          88.3277   70888500.0  623193.979635   
1110    942_24     217     910         108.7050   71995500.0  623849.652027   
1111    942_48     216     907         148.1360   70658500.0  659297.802601   
1112     942_6     216     918         158.8690   58472500.0  582795.558517   

                    
               std  
0     3.535602e+06  
1     3.799654e+06  
2     3.006568e+06  
3     3.379573e+06  
4     4.912602e+06  
...            ...  
1108  4.080307e+06  
1109  3.362987e+06  
1110  3.294163e+06  
1111  3.359265e+06  
1112  2.734257e+06  

[1113 rows x 7 columns]

Note that between peptide and protein data, the peptide frequency (peptideAbundance) and the protein frequency (essentially NPX), is the same for some UniProteins (1:1), but not all of them. This is because some proteins have multiple of the same peptide composing it. So some proteins have more peptideAbundance than NPX, which is why the agg() is not equivalent between the same patient visits with the same proteins.

## Reorganize Training DF

##### What Features Do We Need For Predictions
For easier use in the random forest regressor class, the training data is reorganized so that each row represents a sample patient record. Each column is some feature. The features we need are the visit_month, in order to sort, and the UniProtein ids so that we know the frequency of certain proteins in each patients visit. 

Lets look at the clinical test data once again ...

In [186]:
test_clinical

,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0
5,50423_0,0,50423,updrs_2,50423_0_updrs_2,0
6,50423_0,0,50423,updrs_3,50423_0_updrs_3,0
7,50423_0,0,50423,updrs_4,50423_0_updrs_4,0
8,3342_6,6,3342,updrs_1,3342_6_updrs_1,6
9,3342_6,6,3342,updrs_2,3342_6_updrs_2,6


In [187]:
train_proteins

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


We want to predict the updrs_1, updrs_2, updrs_3, and updrs_4 scores for patients 3342 and 50423 for months 0 and 6. 

In [188]:
# All we need is the patient, month, and the NPX scores of all the UniProteins
def create_prot_df(prot_df):
    records = []   # Stores the list of features for each training sample (each row of the df)
    for grp_idx, data in prot_df:    # grp_idx = grouped index tuple of (patient_id, visit_month), data = a row of the df
        feats = list(grp_idx)
        # For each row, we check a list of all proteins. If any protein is in that sample row (data), we add it that rows list of features
        # Else, we add 0 (that protein doesn't exist for this patient_id)
        for j in range(len(all_prot)):
            if all_prot[j] in data['UniProt'].values: 
                feats.append(data[data.UniProt == all_prot[j]].NPX.iloc[0])
            else:
                feats.append(0.0)
        records.append(feats)
    # Create the dataframe w/ patient, month, and NPX scores of all proteins that each patient_id has > 0
    df = pd.DataFrame(records, columns=['patient_id', 'visit_month'] + all_prot)
    return df

In [189]:
# Sorted list of uniprot ids
all_prot = sorted(set(train_proteins.UniProt).union(set(train_peptides.UniProt)))  # 227 proteins total
# Group protein data by patient_id and month
prot_grp = train_proteins.groupby(["patient_id", "visit_month"])

In [190]:
# Create the protein feature df
df_npx = create_prot_df(prot_grp)
df_npx

,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,55,0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,...,365475.0,35528.00,97005.6,23122.5,60912.6,408698.0,0.0,29758.8,23833.7,18953.5
1,55,6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,...,405676.0,30332.60,109174.0,23499.8,51655.8,369870.0,0.0,22935.2,17722.5,16642.7
2,55,12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,66963.1,...,303953.0,43026.20,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
3,55,36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,67588.6,...,303597.0,48188.40,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
4,942,6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,43423.1,...,253373.0,27431.80,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,64674,84,0.0,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,29248.7,...,260021.0,7139.93,104277.0,10500.0,21944.2,136725.0,62217.5,0.0,10287.7,13848.2
1109,65043,0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,117884.0,...,186414.0,25897.80,0.0,21480.7,57364.0,416142.0,37584.6,0.0,28346.5,35617.5
1110,65043,12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,108246.0,...,301343.0,22343.40,105626.0,20500.8,54011.2,380072.0,40588.9,0.0,17035.7,37064.2
1111,65043,24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,117204.0,...,300439.0,52143.60,139291.0,19449.2,66569.9,300948.0,36150.4,0.0,21286.3,39587.9


In [191]:
# Decided not to use frequency cutoff since i already lost to much data
FREQ_CUTOFF = 0.0
prot_freq = (df_npx.iloc[:, 2:] > 0).mean(axis = 0)   
print(prot_freq)
prot_select = prot_freq.index[(prot_freq >= FREQ_CUTOFF)].tolist()
# now merge in the UPDRS data from the clinical dataset
# we only merge data between train_clinical and train_proteins if they share the same patient_id + visit_month
clin_cols = ["patient_id", "visit_month", "updrs_1", "updrs_2", "updrs_3", "updrs_4"]
join = df_npx[["patient_id", "visit_month"] + prot_select]\
    .merge(train_clinical[clin_cols], on=["patient_id", "visit_month"])
join

O00391    0.686433
O00533    0.999102
O00584    0.988320
O14498    0.927224
O14773    0.940701
            ...   
Q9UHG2    1.000000
Q9UKV8    0.814915
Q9UNU6    0.683738
Q9Y646    0.945193
Q9Y6R7    0.938005
Length: 227, dtype: float64


,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,updrs_1,updrs_2,updrs_3,updrs_4
0,55,0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,...,60912.6,408698.0,0.0,29758.8,23833.7,18953.5,10.0,6.0,15.0,NaN
1,55,6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,...,51655.8,369870.0,0.0,22935.2,17722.5,16642.7,8.0,10.0,34.0,NaN
2,55,12,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,66963.1,...,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9,10.0,10.0,41.0,0.0
3,55,36,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,67588.6,...,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1,17.0,18.0,51.0,0.0
4,942,6,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,43423.1,...,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2,8.0,2.0,21.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,64674,84,0.0,190487.0,24907.9,18543.1,10124.90,2308.71,62095.4,29248.7,...,21944.2,136725.0,62217.5,0.0,10287.7,13848.2,11.0,15.0,45.0,4.0
1064,65043,0,13472.4,927954.0,42661.5,43663.2,20071.30,3278.88,266339.0,117884.0,...,57364.0,416142.0,37584.6,0.0,28346.5,35617.5,2.0,6.0,16.0,NaN
1065,65043,12,14134.9,984651.0,28990.8,42440.9,25357.40,3267.66,270575.0,108246.0,...,54011.2,380072.0,40588.9,0.0,17035.7,37064.2,4.0,7.0,14.0,0.0
1066,65043,24,14659.5,1062020.0,46440.4,38293.0,21971.80,3990.34,221358.0,117204.0,...,66569.9,300948.0,36150.4,0.0,21286.3,39587.9,4.0,8.0,NaN,0.0


In [192]:
df_proteins_0 = join.loc[(join['visit_month'] == 0)]
df_proteins_0 = df_proteins_0.drop('updrs_4', axis=1)
df_proteins_3 = join.loc[(join['visit_month'] == 3)]
df_proteins_3 = df_proteins_3.drop('updrs_4', axis=1)
df_proteins_6 = join.loc[(join['visit_month'] == 6)]
df_proteins_6 = df_proteins_6.drop('updrs_4', axis=1)
common_rows = pd.merge(df_proteins_0, df_proteins_6, on='patient_id')
common_rows

,patient_id,visit_month_x,O00391_x,O00533_x,O00584_x,O14498_x,O14773_x,O14791_x,O15240_x,O15394_x,...,Q9UBR2_y,Q9UBX5_y,Q9UHG2_y,Q9UKV8_y,Q9UNU6_y,Q9Y646_y,Q9Y6R7_y,updrs_1_y,updrs_2_y,updrs_3_y
0,55,0,11254.30,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,...,23499.80,51655.8,369870.0,0.0,22935.2,17722.50,16642.7,8.0,10.0,34.0
1,4161,0,12336.20,1074750.0,25027.3,52936.4,0.00,2180.17,249566.0,45125.8,...,11221.90,23230.0,365088.0,0.0,0.0,0.00,23840.1,1.0,2.0,0.0
2,5645,0,9846.73,539888.0,36067.8,26773.5,43874.00,0.00,174718.0,64748.3,...,23973.80,31343.8,242057.0,34444.9,27677.6,30311.80,9024.8,7.0,11.0,25.0
3,5742,0,7405.52,223579.0,19316.2,27151.9,0.00,2550.78,91819.3,0.0,...,9302.28,26005.7,227630.0,46723.3,18423.8,9144.79,10893.1,6.0,13.0,56.0
4,6211,0,0.00,187035.0,18411.9,19501.2,4420.69,1627.70,39372.2,13175.4,...,7600.02,23326.5,158637.0,0.0,0.0,9283.77,11768.7,6.0,11.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,61503,0,11248.40,710658.0,28116.3,24842.7,20251.90,2004.91,157261.0,43692.1,...,15763.10,32893.5,188276.0,73591.8,15222.4,19682.00,16264.1,1.0,7.0,27.0
64,62723,0,0.00,466328.0,26596.2,19756.0,7752.63,2649.08,107580.0,41415.9,...,13273.40,32026.6,230125.0,67565.9,0.0,17826.10,14096.9,8.0,2.0,8.0
65,62792,0,15119.20,1038820.0,42199.0,39770.3,20818.00,3021.31,286860.0,98972.1,...,21313.90,49049.3,383056.0,0.0,0.0,21886.90,45358.2,0.0,3.0,7.0
66,64669,0,10806.50,491365.0,26262.8,26712.7,15372.50,2995.08,73504.6,57807.2,...,13971.40,35030.0,144959.0,58798.5,0.0,17526.60,19882.2,17.0,20.0,33.0


In [193]:
# There isnt enough data in month 3 to use for month 6 predictions
df_proteins_3

,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,updrs_1,updrs_2,updrs_3
900,56538,3,13270.4,200480.0,36806.7,28861.1,18447.3,3819.43,26619.0,38232.5,...,15364.2,34295.8,166577.0,103992.0,0.0,17513.2,28001.4,6.0,5.0,25.0


In [194]:
# Split the common_rows df back into months 0 and months 6

# Get visit_id column
visit_id_df = common_rows.iloc[:,0]

# Split the common_rows back to month 0 and 6 respectively
df_proteins_0 = common_rows.loc[:, common_rows.columns.str.endswith('x')]
df_proteins_6 = common_rows.loc[:, common_rows.columns.str.endswith('y')]

# Merge visit_id back to each proteins dataframe
# Also strip the trailing strings from the column labels
df_proteins_0 = pd.concat([visit_id_df, df_proteins_0], axis=1)
df_proteins_6 = pd.concat([visit_id_df, df_proteins_6], axis=1)
df_proteins_0.columns = df_proteins_0.columns.str.rstrip('_x')
df_proteins_6.columns = df_proteins_6.columns.str.rstrip('_y')

# Shared visit id of month 0
df_proteins_0

,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,updrs_1,updrs_2,updrs_3
0,55,0,11254.30,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,...,23122.50,60912.6,408698.0,0.0,29758.8,23833.70,18953.50,10.0,6.0,15.0
1,4161,0,12336.20,1074750.0,25027.3,52936.4,0.00,2180.17,249566.0,45125.8,...,9602.96,18350.4,543219.0,67290.7,0.0,13029.40,35091.00,6.0,1.0,0.0
2,5645,0,9846.73,539888.0,36067.8,26773.5,43874.00,0.00,174718.0,64748.3,...,21376.60,36594.1,315995.0,31557.5,25243.0,36819.80,9909.75,12.0,7.0,17.0
3,5742,0,7405.52,223579.0,19316.2,27151.9,0.00,2550.78,91819.3,0.0,...,8371.27,24024.0,169177.0,53700.1,16306.8,5454.99,8131.81,5.0,7.0,38.0
4,6211,0,0.00,187035.0,18411.9,19501.2,4420.69,1627.70,39372.2,13175.4,...,0.00,25247.7,164970.0,35214.0,0.0,13859.90,13141.60,5.0,11.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,61503,0,11248.40,710658.0,28116.3,24842.7,20251.90,2004.91,157261.0,43692.1,...,11701.00,33642.4,202296.0,0.0,16702.4,24882.00,18060.80,3.0,5.0,20.0
64,62723,0,0.00,466328.0,26596.2,19756.0,7752.63,2649.08,107580.0,41415.9,...,16454.40,31906.4,177486.0,69520.8,11264.7,20154.30,14874.10,6.0,2.0,4.0
65,62792,0,15119.20,1038820.0,42199.0,39770.3,20818.00,3021.31,286860.0,98972.1,...,20480.80,48041.4,394514.0,70445.0,0.0,17492.70,29112.00,0.0,2.0,10.0
66,64669,0,10806.50,491365.0,26262.8,26712.7,15372.50,2995.08,73504.6,57807.2,...,15743.50,27776.3,171664.0,63783.2,0.0,0.00,25094.20,12.0,14.0,27.0


In [195]:
# Shared visit id of month 6
df_proteins_6

,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,updrs_1,updrs_2,updrs_3
0,55,6,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,...,23499.80,51655.8,369870.0,0.0,22935.2,17722.50,16642.7,8.0,10.0,34.0
1,4161,6,11191.5,836088.0,18232.4,44420.0,0.00,3727.25,213213.0,31698.5,...,11221.90,23230.0,365088.0,0.0,0.0,0.00,23840.1,1.0,2.0,0.0
2,5645,6,13070.0,608796.0,34771.8,0.0,32705.30,2717.30,185609.0,39250.5,...,23973.80,31343.8,242057.0,34444.9,27677.6,30311.80,9024.8,7.0,11.0,25.0
3,5742,6,0.0,398967.0,23080.5,30432.5,0.00,1006.44,126966.0,0.0,...,9302.28,26005.7,227630.0,46723.3,18423.8,9144.79,10893.1,6.0,13.0,56.0
4,6211,6,0.0,183667.0,13686.4,16421.7,6851.54,2188.71,44039.4,12870.0,...,7600.02,23326.5,158637.0,0.0,0.0,9283.77,11768.7,6.0,11.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,61503,6,11674.3,724274.0,24918.9,26372.7,21431.70,924.61,165806.0,72638.7,...,15763.10,32893.5,188276.0,73591.8,15222.4,19682.00,16264.1,1.0,7.0,27.0
64,62723,6,0.0,446637.0,25861.0,25126.2,19681.20,3362.51,125738.0,40699.2,...,13273.40,32026.6,230125.0,67565.9,0.0,17826.10,14096.9,8.0,2.0,8.0
65,62792,6,13772.0,771691.0,41596.5,39963.9,23437.20,2160.20,237267.0,87903.0,...,21313.90,49049.3,383056.0,0.0,0.0,21886.90,45358.2,0.0,3.0,7.0
66,64669,6,0.0,364638.0,23734.6,20410.8,12074.70,2864.83,70720.8,35008.5,...,13971.40,35030.0,144959.0,58798.5,0.0,17526.60,19882.2,17.0,20.0,33.0


## Training DataFrame
df_proteins_0 is the common visit id's from month 0 and df_proteins_6 are the same patient visits in month 6. Therefore, I will train the model with the month 0 protein npx scores and predict the updrs on month 6. I need to take the protein data from df_proteins_0 and split it to test and train data. I will do this manually as to not test updrs scores of mixed patient id's. I will also repeat this with the updrs columns in df_protein_6 which will be y_train and y_test.

In [280]:
### 68 rows means 55 training rows and 13 test rows for a  0.85:0.15 train test split.
X_train = df_proteins_0.sample(n=58, random_state=10)
X_test = df_proteins_0.drop(X_train.index)
y_train = df_proteins_0.sample(n=58, random_state=10)
y_test = df_proteins_0.drop(y_train.index)

# Check if the split worked
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

if X_train['patient_id'].equals(y_train['patient_id']): # Is the rows of the features X and targets y in the same order
    print("Train rows are the same order")
if X_test['patient_id'].equals(y_test['patient_id']):
    print("Test rows are the same order")
    
# Grab the columns we want
X_train = X_train.iloc[:,2:-3]
X_test = X_test.iloc[:,2:-3]
y_train = y_train.iloc[:,-3:]
y_test = y_test.iloc[:,-3:]

58
10
58
10
Train rows are the same order
Test rows are the same order


At this point X_train, y_train, X_test, and y_test should have the features needed to train. A lot of samples have been lost during the dataframe cleanup so the results may not be very good.

In [281]:
# Mean updrs scores (1 through 4) for each month
updrs_means = train_clinical.groupby("visit_month")[["updrs_1", "updrs_2", "updrs_3", "updrs_4"]].mean()
updrs_means

,updrs_1,updrs_2,updrs_3,updrs_4
visit_month,,,,
0,5.572581,4.439516,13.754032,2.047619
3,5.513043,6.634783,20.482456,0.142857
6,7.109375,6.963542,20.272251,2.307692
9,6.080808,7.204082,20.474747,0.488372
12,6.222222,5.320988,16.165975,1.308271
18,7.299465,6.866310,19.016216,1.218978
24,6.670782,5.658436,16.489540,1.658065
30,8.231214,7.647399,21.710983,1.753425
36,7.331858,6.230088,18.316964,1.765432


## Import Model Libraries

In [282]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.metrics import accuracy_score
import math

## Random Forest Classifier vs Bagging Classifier Background Info
Both ensemble methods utilize multiple decision trees to perform classification. However, they differ in a few ways. 

#### Random Forest
In random forest classification, a random subset of features is selected for each iteration of the test_train_split. Random forests sample without replacement, meaning each sample instance can be selected only once per sample. Finally, when the training is done, the majority vote is based on a subset of randomly selected decision trees, rather than all the decision trees. This is done to reduce overfitting on the training data.

#### Bagging
On the other hand, bagging is slightly less robust than random forest classification. Instead of generalizing the model, in bagging, sample instances are selected with replacement. This means that there is a chance that a training sample can have instances of the same data. E.g. Given a set {0, 1, 2}, in random forest classifiers, a sample has to have unique instances, like {1, 2, 0}, {0, 2, 1}, etc. But in bagging, there is replacement so we could have {0, 0, 1} or event {2, 2, 2}. In addition, when we reach the end of training, the final decision is based on the output of every decision tree, which may cause overfitting.

## Random Forest Regressor Training

In [283]:
def replace_zeros_with_mean(col):
    mean = col[col != 0].mean()  # calculate the mean, excluding 0's
    return col.replace(0, mean)  # replace 0's with the mean

# apply the function to each column using apply()
# X_train = X_train.apply(replace_zeros_with_mean)
# X_test = X_test.apply(replace_zeros_with_mean)

In [284]:
X_train.head()

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
21,12012.20,537174.0,30288.1,34752.3,26889.6,3129.43,208436.0,56576.1,217932.0,119002.0,...,462003.0,13651.3,97110.2,22201.20,27724.7,337200.0,59605.1,17880.5,13882.90,0.00
2,9846.73,539888.0,36067.8,26773.5,43874.0,0.00,174718.0,64748.3,265121.0,187226.0,...,0.0,45517.2,117741.0,21376.60,36594.1,315995.0,31557.5,25243.0,36819.80,9909.75
32,10878.10,760421.0,33891.6,36782.8,19534.9,3515.34,100097.0,44225.7,325065.0,45852.4,...,262684.0,60229.0,117840.0,0.00,46144.1,273488.0,116767.0,0.0,21232.40,23684.20
3,7405.52,223579.0,19316.2,27151.9,0.0,2550.78,91819.3,0.0,101594.0,154196.0,...,409937.0,22499.7,94190.3,8371.27,24024.0,169177.0,53700.1,16306.8,5454.99,8131.81
6,11966.00,684118.0,36325.0,26503.1,14515.0,3822.82,168925.0,67797.5,219225.0,202070.0,...,209758.0,46035.6,112157.0,12466.00,54272.9,251016.0,62934.6,24668.7,24942.30,19453.60


In [285]:
X_test.head()

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
9,0.0,276293.0,23664.0,28587.1,17398.2,2626.47,63699.8,45855.0,150007.0,147000.0,...,333379.0,13537.9,85563.3,14365.4,34041.0,233758.0,65618.5,0.0,16574.8,8240.38
15,13636.9,577417.0,30478.1,35489.4,24659.0,2800.74,128441.0,57095.5,208353.0,99350.1,...,395821.0,35409.0,126979.0,19627.9,28884.0,223221.0,104695.0,0.0,20951.2,36622.40
25,0.0,325310.0,37043.1,14729.6,19633.6,2977.11,60330.0,28566.7,140056.0,145996.0,...,324832.0,40184.4,87045.1,15538.7,32803.2,167567.0,0.0,13803.3,19726.3,37786.90
28,12256.2,363294.0,20434.0,19796.7,15733.5,6191.27,50754.7,42690.1,159675.0,92164.5,...,520519.0,33206.8,105552.0,12686.4,34226.2,126232.0,85433.3,0.0,14485.9,14134.90
29,0.0,477149.0,32904.2,34663.3,33321.8,3802.17,176256.0,55436.6,190096.0,111336.0,...,290302.0,32568.4,77823.9,19224.7,35705.7,310932.0,72278.6,34348.1,19562.3,39242.70


In [286]:
y_train.head()

,updrs_1,updrs_2,updrs_3
21,5.0,5.0,23.0
2,12.0,7.0,17.0
32,8.0,5.0,12.0
3,5.0,7.0,38.0
6,2.0,2.0,13.0


In [287]:
y_test.head()

,updrs_1,updrs_2,updrs_3
9,6.0,3.0,16.0
15,1.0,2.0,15.0
25,7.0,22.0,41.0
28,6.0,15.0,18.0
29,9.0,8.0,20.0


Change the target dataframes so that they are quantized between 25% percentiles. Find the min and max of each updrs score. Then, take the difference and break into 4 percentiles. Finally, go through each updrs column and replace the score with what percentile it lies in (0,1,2,3). When training the data, we take the target an place it into a percentile with classification rather than using regression. This may help to improve our accuracy score because we can better ignore outliers.

In [288]:
# iterate through each column
def quantize_df(df):
    for col in df.columns:
        # get the min and max values for this column
        col_min = df[col].min()
        col_max = df[col].max()
        # calculate the thresholds for each level
        q1 = col_max * 0.25
        q2 = col_max * 0.5
        q3 = col_max * 0.75
        print(q1, q2, q3)
        # iterate through each value in this column
        for index, value in df[col].iteritems():
            # apply the logic you described
            if value <= q1:
                df.at[index, col] = 0
            elif value <= q2:
                df.at[index, col] = 1
            elif value <= q3:
                df.at[index, col] = 2
            else:
                df.at[index, col] = 3

In [289]:
quantize_df(y_test)
quantize_df(y_train)

2.25 4.5 6.75
5.5 11.0 16.5
10.25 20.5 30.75
4.5 9.0 13.5
5.0 10.0 15.0
11.5 23.0 34.5


In [290]:
y_test.head()

,updrs_1,updrs_2,updrs_3
9,2.0,0.0,1.0
15,0.0,0.0,1.0
25,3.0,3.0,3.0
28,2.0,2.0,1.0
29,3.0,1.0,1.0


In [291]:
y_train.head()

,updrs_1,updrs_2,updrs_3
21,1.0,0.0,1.0
2,2.0,1.0,1.0
32,1.0,0.0,1.0
3,1.0,1.0,3.0
6,0.0,0.0,1.0


#### Validate the dataframes are properly synced

In [292]:
# X_train and y_train should have same matrix # rows. The same for _test
if X_train.shape[0] == y_train.shape[0]:
    print("Train Data: Success")
if X_test.shape[0] == y_test.shape[0]:
    print("Test Data: Success")

Train Data: Success
Test Data: Success


In [293]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Since my target outputs are quantized values, I need to use a classifier rather than a regressor for my random forest. 

## UPDRS 1

In [296]:
from sklearn.model_selection import GridSearchCV

# Grab only the updrs targets
y_train_1 = y_train['updrs_1']
y_test_1 = y_test['updrs_1']

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [0.1, 0.2, 0.3]
}

# Create the RF model and perform grid search
rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train_1)

# Print the best hyperparameters and cross-validation score
print("Best hyperparameters:", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_*100} %")

# Fit the model with the best hyperparameters
model1 = grid_search.best_estimator_
model1.fit(X_train, y_train_1)

# Make predictions and calculate the accuracy of the model for the test and train data
y_pred_1 = model1.predict(X_test)
y_pred_2 = model1.predict(X_train)
accuracy = accuracy_score(y_test_1, y_pred_1)
accuracy2 = accuracy_score(y_train_1, y_pred_2)
print(f"Accuracy: {accuracy*100} %")
print(f"Accuracy: {accuracy2*100} %")

Best hyperparameters: {'max_features': 'sqrt', 'min_samples_split': 0.1, 'n_estimators': 200}
Best cross-validation score: 46.36363636363637 %
Accuracy: 20.0 %
Accuracy: 100.0 %


We are overfitting to the training data too well. I tried to add gridsearch cv to find the best hyperparameters for the RF classifier. Then I decreased the min_samples_split, and lowered the number of estimators to conform the training data less. I was averaging a best cross-validation score of ~50 %. I tried to split the training and test data with a 0.7:0.3 ratio instead of 0.8:0.2 to hopefully train the model a little less, but it actually got slightly worse. I guess it is because I dont have much data to train on in the first place.

## UPDRS 2

In [297]:
# Grab only the updrs targets
y_train_2 = y_train['updrs_2']
y_test_2 = y_test['updrs_2']

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [0.1, 0.2, 0.3]
}

# Create the RF model and perform grid search
rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train_2)

# Print the best hyperparameters and cross-validation score
print("Best hyperparameters:", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_*100} %")

# Fit the model with the best hyperparameters
model2 = grid_search.best_estimator_
model2.fit(X_train, y_train_2)

# Make predictions and calculate the accuracy of the model for the test and train data
y_pred_1 = model2.predict(X_test)
y_pred_2 = model2.predict(X_train)
accuracy = accuracy_score(y_test_2, y_pred_1)
accuracy2 = accuracy_score(y_train_2, y_pred_2)
print(f"Accuracy: {accuracy*100} %")
print(f"Accuracy: {accuracy2*100} %")

C:\Users\mkouc\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'max_features': 'sqrt', 'min_samples_split': 0.3, 'n_estimators': 50}
Best cross-validation score: 58.484848484848484 %
Accuracy: 40.0 %
Accuracy: 86.20689655172413 %


## UPDRS 3

In [299]:
# Grab only the updrs targets
y_train_3 = y_train['updrs_3']
y_test_3 = y_test['updrs_3']

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [0.1, 0.2, 0.3]
}

# Create the RF model and perform grid search
rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train_3)

# Print the best hyperparameters and cross-validation score
print("Best hyperparameters:", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_*100} %")

# Fit the model with the best hyperparameters
model3 = grid_search.best_estimator_
model3.fit(X_train, y_train_3)

# Make predictions and calculate the accuracy of the model for the test and train data
y_pred_1 = model1.predict(X_test)
y_pred_2 = model1.predict(X_train)
accuracy = accuracy_score(y_test_3, y_pred_1)
accuracy2 = accuracy_score(y_train_3, y_pred_2)
print(f"Accuracy: {accuracy*100} %")
print(f"Accuracy: {accuracy2*100} %")

C:\Users\mkouc\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'max_features': 'sqrt', 'min_samples_split': 0.1, 'n_estimators': 100}
Best cross-validation score: 56.96969696969697 %
Accuracy: 40.0 %
Accuracy: 32.758620689655174 %
